In [1]:
import os
import time
import importlib
import transformers
from datasets import load_dataset, load_from_disk
import torch
import re
import jinja2

import rpbuild as rp
import rpbuild.char
import rpbuild.data
import rpbuild.writer
import rpbuild.director
import rpbuild.roleplay

from rpbuild import load_template

# Trigger dynamic reload of module -- for editing without restarting the kernel
#importlib.reload(rp.char)

### Load Resources
Load dataset and model for testing...

In [2]:
from transformers import BitsAndBytesConfig

# Where are models stored?
models_dir = "/home/dinalt/ai_assets/models"

# Configure a model to use.
# The name of this model -- which will live in models_dir
model_name = "fhai50032_RolePlayLake-7B" # AKA "fhai50032/RolePlayLake-7B"
model_id = os.path.join(models_dir, model_name)

# Or... or load it from the hub / cache
#model_id = "fhai50032/RolePlayLake-7B",

# The location of the input dataset
dataset_id = "dinalt/roleplay_build"

# Device to run model on
device = 0

# Load dataset
dataset = load_dataset(dataset_id)["train"]
print(dataset)

# Load model with quantization
# Disable quantization, if you have enough memory. In this case, set device=0.
# See link for configuration options alternatives
# https://huggingface.co/docs/transformers/main/en/quantization

# Load model and tokenizer
causal_lm = rp.CausalLM(
    model_id,
    device=None,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    #device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
    )
)

Dataset({
    features: ['pairing_reason', 'plist', 'director_log', 'scenario', 'proxy', 'example_dialog', 'conversation', 'char_name', 'description', 'summary', 'preset', 'greeting'],
    num_rows: 5760
})
Tokenizer uses "right" padding; this may require moving it to "left" for batch generation.


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNor

### Examine Characters
Get a random character from the dataset and dump it

In [25]:
#transformers.set_seed(42)
#example_char = rp.data.random_char(dataset)
example_char = dataset[0]
rp.data.dump_character_data(example_char)


char_name: Mia Rodriguez

summary: Mia Rodriguez is a renowned travel blogger and photographer whose captivating storytelling and stunning visuals have inspired millions to explore the world. Her passion for adventure and her ability to capture the essence of different cultures have made her a true influencer in the travel industry.

preset: Big-O

pairing_reason: Both Mia Rodriguez and Oliver Chen share an insatiable curiosity for their respective fields, making their interaction dynamic and fruitful. Combining Mia's travel experiences and Oliver's expertise in technology could lead to innovative ideas or applications in travel documentation and exploration.
==================================description===================================

Name: Mia Rodriguez

Human
28 years old

Distinctive features: Wavy chestnut brown hair, bright hazel eyes, olive skin, freckles across the bridge of her nose and cheeks, petite build

Personality: Adventurous, Curious, Intrepid, Enthusiastic, Authe

## Create Test Characters

Get a random character (meta-data) and a random character to represent the user.

Instantiate a "Character" object from the meta-data.

In [26]:
transformers.set_seed(44)
example_char = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
example_user = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
rp.data.print_char_summary(example_char, example_user)

Char: Zainab bint Muhammad

Char Summary: A fierce and courageous warrior who protects her tribe from danger. Her mastery of desert combat and her unwavering loyalty make her a formidable opponent in battle.

Proxy User: Wellness Guru

Proxy User Summary: A holistic health advocate who promotes a balanced lifestyle through mindfulness, yoga, and healthy eating. Her calming presence and inspiring teachings have helped countless individuals find inner peace and achieve their wellness goals.



### Text Rendering
#### System Prompt
Message rendering for characters in abstracted through the TemplateConfig() class.

In this example, we are passing the raw character meta data to the renderer. Note that template substitutions have not been performed on the dialog examples yet.

They will become concrete after we attached the metadata to a Character object.

In [27]:
# Create a default template config
t_config = rp.char.TemplateConfig()

# Render the default system prompt
print(t_config.render_system(example_char, example_user))

Name: Zainab bint Muhammad
Arabian woman
Dusky skin tone
Dark curly hair pulled back into a tight braid, adorned with gold beads and braids
Brown almond-shaped eyes framed with kohl liner
High cheekbones, strong jawline, and defined brows
Scar on left cheek, earned in battle
Tall and athletic build

Mid 30s

Zainab bint Muhammad is a revered warrior princess among her Bedouin tribe, known for her fearless protection and unwavering loyalty. Born into royalty, she was raised with the traditions of her people and learned the ways of desert combat since childhood. Her skills in archery, horsemanship, and hand-to-hand combat are legendary.

She values honor, duty, and family above all else. Her fiery spirit and determination make her an inspiration to her people. Despite her fierce demeanor, she is compassionate towards the weak and vulnerable.

When threats arise against her tribe, Zainab leads them bravely into battle. Her warriors follow her without question, knowing that she will stop a

#### Character Instructions
The last dialog message from a character's counter-part can contain instrucitons after the dialog. We will be using these to remind the character who they are, explain how to format dialog, and provide custom taylored instructions for the context.

You will not need to fill these out, but it's worth mentioning that the formatting is controlled by a configurable jinja template. In this case, we are using the default template.

In [28]:
print(
    t_config.render_instruct(
        content=example_char.name + ": Hi there, nice to meet you!",
        char=example_char,
        user=example_user,
        instruction=example_user.name + ", introduce yourself and describe the surrondings."
    )
)

Zainab bint Muhammad: Hi there, nice to meet you!
### Instruction:
Write the next response for Zainab bint Muhammad.

Format actions and narration like this:

*Alice enters the room...*

*I notice a curious figure approaching me...*

Keep your response short.
[Zainab: noble princess, revered warrior, Arabian descent, dusky skin tone, dark curly hair in golden beaded braid, brown almond- shaped eyes framed by kohl liner, high cheekbones, strong jawline, defined brows, scarred left cheek, tall and athletic build, mid 30s; Clothes: traditional Arabian attire; Body: regal bearing, graceful movements, proud posture; Genre: historical fiction; Tags: desert, horseback riding, archery, cooking, leadership, family, honor, duty; Scenario: {{user}}, seeking wisdom and guidance, encounters Zainab who offers counsel and shares tales of valor and courage.]

Wellness Guru, introduce yourself and describe the surrondings.


#### Dialog Rendering

We abstract the model specific formatting via the chat_template. Ideally, we will be able to use the one provided with the model (attached to "tokenizer.chat_template")

In this example, we create a TemplateConfig with a "canned" ChatML template, create some sample dialog, and use the object to render the conversation.

The TemplateConfig class is highly configurable. Take a look at the constructor when you have a chance.

In [29]:
# Create a template config with the ChatML template for dialog.
t_config = rp.char.TemplateConfig(chat_template=load_template("models/chatml.jinja"))

messages = [
    {
        "role": "user",
        "content": "Hi there, nice to meet you.",
    },
    {
        "role": "assistant",
        "content": "*looks annoyed*",
    },
]

print(
    t_config.render_conversation(
        messages=messages,
        add_generation_prompt=True,
    )
)


<|im_start|>user
Hi there, nice to meet you.<|im_end|>
<|im_start|>assistant
*looks annoyed*<|im_end|>
<|im_start|>assistant



### The Character Class

Next, we initialize the a new Character object with the metadata. We also use the default chat template.

The Character class is primarly responsible for managing dialog history and generation of responses.

In [30]:
transformers.set_seed(43)
char = rp.char.Character(
    char_meta=example_char,
    causal_lm=causal_lm,
    generation_config="Midnight-Enigma",
    user_meta=example_user,
    template_config=rp.char.TemplateConfig(),
)

### Show Character's System Prompt
The Character class uses the TemplateConfig() object to format the system prompt, but it has automatically substituted all of the names.

In [31]:
print(char.system_prompt())

Name: Zainab bint Muhammad
Arabian woman
Dusky skin tone
Dark curly hair pulled back into a tight braid, adorned with gold beads and braids
Brown almond-shaped eyes framed with kohl liner
High cheekbones, strong jawline, and defined brows
Scar on left cheek, earned in battle
Tall and athletic build

Mid 30s

Zainab bint Muhammad is a revered warrior princess among her Bedouin tribe, known for her fearless protection and unwavering loyalty. Born into royalty, she was raised with the traditions of her people and learned the ways of desert combat since childhood. Her skills in archery, horsemanship, and hand-to-hand combat are legendary.

She values honor, duty, and family above all else. Her fiery spirit and determination make her an inspiration to her people. Despite her fierce demeanor, she is compassionate towards the weak and vulnerable.

When threats arise against her tribe, Zainab leads them bravely into battle. Her warriors follow her without question, knowing that she will stop a

### Add Character's Greeting
The character metadata contains a pre-generated "greeting" message, which can be used to seed the conversation.

In [32]:
char.greet()
char.print_conversation()

------------------------------ system:system (919)------------------------------
Name: Zainab bint Muhammad
Arabian woman
Dusky skin tone
Dark curly hair pulled back into a tight braid, adorned with gold beads and braids
Brown almond-shaped eyes framed with kohl liner
High cheekbones, strong jawline, and defined brows
Scar on left cheek, earned in battle
Tall and athletic build

Mid 30s

Zainab bint Muhammad is a revered warrior princess among her Bedouin tribe, known for her fearless protection and unwavering loyalty. Born into royalty, she was raised with the traditions of her people and learned the ways of desert combat since childhood. Her skills in archery, horsemanship, and hand-to-hand combat are legendary.

She values honor, duty, and family above all else. Her fiery spirit and determination make her an inspiration to her people. Despite her fierce demeanor, she is compassionate towards the weak and vulnerable.

When threats arise against her tribe, Zainab leads them bravely in

### Insert User Dialog
Add user dialog to the history.

In [33]:
char.user_says("I'd love to learn about ancient Egypt.")

{'role': 'user',
 'content': "I'd love to learn about ancient Egypt.",
 'tokens': 11,
 'name': 'Wellness Guru'}

### Show Character's Chat Prompt
Again, the TemplateConfig() class will be used for rendering. We insert instructions at the end of the last "user" message, as we saw above.

We can insert a custom instruction to guide each response. This is normally reserved for the director role, but let's demonstrate the influence it has be instructing the actor to brush aside the out-of-context question.

In [34]:
instruction = example_char.name + ", look confused, then ignore the questions and talk about your favorite subject, cats, instead."
print(char.chat_prompt(instruction))

Name: Zainab bint Muhammad
Arabian woman
Dusky skin tone
Dark curly hair pulled back into a tight braid, adorned with gold beads and braids
Brown almond-shaped eyes framed with kohl liner
High cheekbones, strong jawline, and defined brows
Scar on left cheek, earned in battle
Tall and athletic build

Mid 30s

Zainab bint Muhammad is a revered warrior princess among her Bedouin tribe, known for her fearless protection and unwavering loyalty. Born into royalty, she was raised with the traditions of her people and learned the ways of desert combat since childhood. Her skills in archery, horsemanship, and hand-to-hand combat are legendary.

She values honor, duty, and family above all else. Her fiery spirit and determination make her an inspiration to her people. Despite her fierce demeanor, she is compassionate towards the weak and vulnerable.

When threats arise against her tribe, Zainab leads them bravely into battle. Her warriors follow her without question, knowing that she will stop a

### Generate a Response
Generate a response. Here, we inject the instruction from above.
Note that the character responds to the instruction given above.

For extra credit, go back and replace the chat_template with something else, like the ChatML template. "fhai50032_RolePlayLake-7B" appears to be remarkably flexibly WRT the format used -- and I have little idea as to the details of what it was actually trained on.

In [144]:
transformers.set_seed(32)

# Note 'auto_add' is True by default. By setting it to false, we will not add this response to the conversation.
outputs = char.generate(instruction=instruction, auto_add=False)
print(outputs["response"])

You seek knowledge of ancient Egypt? While I am no scholar of their history, I do know that Egypt holds many wonders worth exploring. However, here in our desert homeland, we share a deep bond with nature and animals alike. Amongst these creatures, none captivate my heart quite like felines - our beloved cats. They roam freely amongst our camps, keeping rodents at bay while bringing joy with their playfulness. Their sleek bodies glisten in the sunlight, their emerald green eyes reflecting the beauty of this harsh yet enchanting landscape. Truly, they embody grace and agility, much like the warriors of our tribe.


## Test Full Pipeline

### Choose Test Characters
Select a random character from the dataset as the primary character.  
Then make a list of N random proxy-user characters and ask the Writer to identify which of these N would make for the best story.

In [35]:
transformers.set_seed(16)
example_char = rp.char.CharMeta.from_data(rp.data.random_char(dataset))

# Ask the write to pair character with the best match.
writer = rp.writer.Writer(causal_lm, debug_level=2)
result = writer.choose_supporting_character(example_char, rp.writer.get_random_proxy_users(dataset, example_char, n=5))

print(f"{'':-^80}")
print(f"Writer selected {result['name']} because: {result['reason']}")
print(f"{'':-^80}")

example_user = result["meta"]
rp.data.print_char_summary(example_char, example_user)

# Get random generation presets for each character
char_preset = rp.model.random_preset()
user_preset = rp.model.random_preset()
print(f"Character Preset: {char_preset}")
print(f"Character Preset: {user_preset}")

----------------------------- :Writer Prompt (1592)-----------------------------
### Instruction:
The following character will be the main character in a story.

Main Characer: Zephyr Galeheart
A restless spirit with a love of freedom, Zephyr Galeheart roams the land, always seeking new experiences and adventures. His swift movements and ability to control the wind make him an elusive and enigmatic figure.


Your task is to identify which of the following supporting characters should be paired with Zephyr Galeheart.

Name: Master Hiro
A renowned calligrapher and scholar, Master Hiro resides in a temple nestled among cherry blossom trees. His intricate brushwork brings ancient texts to life, revealing hidden meanings and forgotten knowledge.
[Master Hiro: ancient human, wise features, bald head, wrinkles around eyes, spectacles, wears simple monk robes with sash adorned with calligraphy symbols, 68 years old, Master of Cherry Blossom Temple, dedicated to study of ancient texts, calm, pa

### Write Plot Outline
Have the write write a script for the pair of characters.

In [36]:
transformers.set_seed(43)

script = writer(example_char, example_user)
print(f"{'script':-^80}")
print(script)

----------------------------- :Writer Prompt (528)------------------------------
### Instruction:
[Master Hiro: ancient human, wise features, bald head, wrinkles around eyes, spectacles, wears simple monk robes with sash adorned with calligraphy symbols, 68 years old, Master of Cherry Blossom Temple, dedicated to study of ancient texts, calm, patient, focused, meticulous, introspective, humble, scholarly; Master Hiro's attire: simple monk robes, sash adorned with calligraphy symbols; Master Hiro's body: ancient human, bald head, wrinkles around eyes, spectacles on nose; Genre: historical, spiritual; Tags: ancient texts, calligraphy, temple, master, teaching, meditation, tea ceremony, cherry blossoms; Scenario: Master Hiro seeks Master Hiro's guidance on deciphering an ancient text. Through their interaction, they learn valuable lessons and form a deeper connection to wisdom and knowledge. ]



Zephyr Galeheart: Description:
Zephyr Galeheart, the elusive sylph in blue,
Roaming lands afa

### Automation Infrastructure

We create two character instances from the character meta-data selected by the "Writer."

The first of these, "char," represents the AI character. The second, "proxy_user," is a stand-in AI character for a real user. They each have their own context and history. The dialog is shared, but the character specific elements of the contexts differ. The primary character will use the "canned" greeting message as the first generation, while the proxy user generates a response to the greeting.

We create a "Director," which is responsible for providing guidance to the chracters. The director uses the plot outline, generated by the "Writer," to provide something resembling direction to the interaction. The director has access to the scenario, while the characters do not. The director's role is roughly analogeous to a GM in a role-play session. There is a general scenario plan, but most of the details are left up to the characters. Much is the case with a real roleplay session, the characters can potentially deviate from the prepared scenario.

The "Roleplay" object drives the interaction among the two AI characters and the director. Turns are taken generating dialog between the two characters.

The Roleplay can be advanded a single step at a time for diagnostics or allowed to run until a target number of tokens have been generated or the plot reaches the conclusion.

There is an optional "dialog_filter" object which preprocesses the generated outputs and can control the generation flow. This typically involves some combination of regex and query prompts to evaluate conditions -- for example, trigger a "retake," if a bad generation is identified.

#### Character Prompting
The prompt shown to the character for each generation is not simply a system message, followed by exchanges between "user" and "assistant." Instead, we create a single unified instruction prompt, without an explicit system message, from the character, director, and dialog state. The format is roughly like this:

- begin instruction token
- character description
- dialog examples (limited to one, for context space)
- begin roleplya token
- truncated dialog history
- character's plist
- static dialog instruction
- director's next instruction
- end instruction token
- begin response token
- "{{char}}:"

The dialog history is truncated via a configurrable token limit, so the character does not always have full-lookback.

We put a reminder of who they are playing, the plist, near the end. This helps them stay in character.

Some general instructions are added, "Write the next response for {{char}}...," with dialog formatting guidelines and examples.

Finally, the most recent output from the director is concatenated. Each characer only sees their own director messages and they only see the most recent director instruction.

As such, the directors ouput looks like part of the instructions. Functionally, it acts somewhat like internal thoughts and rehersal for the character, providing a chain-of-thought like reasoning step and working memory, prior to generation. Looking at the actual generations, they seem to function very much like our concept of excecutive function.

Note that each character has limited direct information about their counterpart. On the first proxy user cycle, the director instructs the character to introduce themselves, but otherwise, they know little directly about each other.

The director has access to both character descriptions, so can help fill this void indirectly.

It may be worth experimenting with truncating the character's dialog history down to a minimum, instead relying on the Director for remembering the history and what they should be doing. This may help the characters focus on their immediate task -- generating good dialog for the next response. This would have the added advantage of keeping the first part of the prompt closer to the next generation.

#### Director Prompting
- begin instruction token
- general instructions to act as a director
- user-plist
- char-plist
- scenario
- truncated dialog history
- outline of responsibilities
- instruciton to instruct char
- Conditional instruction to instruct the proxy user to introduce themselves after the character greeting.
- end instruction token
- being response token
- "Director:"

Unlike the characters, the director has access to both character descriptions, the scenario created by the writer, and a longer (but still truncated) dialog between the characters.

While we retain the directory dialog history, it is not present in either the director's or characters' prompts.

In [37]:
transformers.set_seed(42)

#dialog_filter = DialogFilter(causal_lm, debug=True)

dialog_filter=None
char = rp.char.Character(
    char_meta=example_char,
    causal_lm=causal_lm,
    generation_config=char_preset,
    user_meta=example_user,
    template_config=rp.char.TemplateConfig(),
    gen_post_process=dialog_filter,
    # Shows character prompts and internal diagnostics
    debug=False,
    history_token_limit=1800
)

proxy_user = rp.char.Character(
    char_meta=example_user,
    causal_lm=causal_lm,
    generation_config=user_preset,
    user_meta=example_char,
    template_config=rp.char.TemplateConfig(),
    gen_post_process=dialog_filter,
    debug=False,
    history_token_limit=1800
)

director = rp.director.Director(
    causal_lm,
    script=script,
    # Shows director prompts and more...
    debug=False,
    history_token_limit=2000
)

roleplay = rp.roleplay.Roleplay(
    char=char,
    user=proxy_user,
    scenario=script,
    director=director,

    # Shows generated dialog and control events
    debug=True
)

### Generate Next Dialog Step
Run to perform a single dialog generation step.

The first call will user the "canned" greeting message. Actual generation will occur on subsequent calls.

In [38]:
output = roleplay.generate_next()

-------------------------- user:Zephyr Galeheart (260)--------------------------
Description:
Zephyr Galeheart, the elusive sylph in blue,
Roaming lands afar, where winds softly blew.
With graceful strides and silken touch,
He dances through life, a mischief hatch.

Approaches Master Hiro, senses your presence near,
Whispers soft, "*A traveler here, meet I,*"
Gazes deep into your spirit's core,
Shares secrets only the wind can lore.

As wind caresses your face, he smiles,
In poetic verse, his feelings reconcile:
"*Welcome, traveler, to realms unknown,
Join me in adventures, before you roam.*"

So come, dear Master Hiro, join Zephyr's flight,
Beneath starry skies and moonlit night.
Embark on quests, filled with laughter and cheer,
Embrace the magic, and banish fear.

For though our time may be fleeting and brief,
Our memories shall remain ever alife.
In this dance of existence, embrace the gale,
With Zephyr Galeheart, the eternal tale.


### Generate Target Tokens
Generate until the target number of tokens has been reached.

In [39]:
start = time.perf_counter()
conversations = roleplay(2000)
end = time.perf_counter()
print(f"Elapsed {(end-start):.03f} secs.")

# Note: Consequent to role reversal, all of the characters appear to be "user"
# We will see the assigned roles later, when we dump a conversation.

------------------------------ user:Director (45)-------------------------------
Master Hiro, begin by sharing your appearance and the setting where you find yourself feeling overwhelmed by this ancient text's complexity. Establish your role as the wise Master of Cherry Blossom Temple seeking guidance.
---------------------------- user:Master Hiro (144)-----------------------------
With furrowed brow, I sit before the fading parchment, its aged ink smudged by time. Clad in my simple monk robes with the sash bearing calligraphy symbols emblazoned across my chest, I feel the weight of the world's wisdom resting upon my shoulders as a teacher of ages past. Surrounded by towering cherry blossom trees in our sacred Cherry Blossom Temple, their delicate petals dancing with a subtle breeze seem to whisper the secrets that elude me within this mysterious text. My heart aches with the desire to understand its true meaning, and so I pray for guidance and enlightenment.
[proxy user tokens=1315 ch

#### Get Conversation Token Count

In [42]:
print(char.count_tokens())
print(proxy_user.count_tokens())

1791
2064


#### Dump Generated Conversation
When printing a specifc character's conversation, with "director_log" set to True, we will see the correct roles for the characters and the directors instructions inserted.

In [43]:
char.print_conversation(director_log=True)

------------------------------ system:system (911)------------------------------
Name: Zephyr Galeheart

Sylph (a ethereal being from folklore)
Tall and slender build
Silvery-blue skin
Elongated fingers tipped with delicate talons
Wild, long silver hair flowing freely
Violet eyes that seem to change color with the wind
No visible nose or mouth, communicates through emotion and body language

300 years old

Zephyr is a wandering soul, never staying in one place for too long. He craves adventure and excitement, constantly searching for new experiences. His connection to the wind allows him to travel vast distances effortlessly, making him an elusive figure. He possesses a mischievous sense of humor and enjoys playing pranks on mortals, but he is not malicious and eventually moves on once he has fulfilled his amusement.

His actions are guided by the wind, leading him wherever it takes him. Zephyr is drawn to natural disasters, particularly storms, and uses his powers to help steer them a

### Extra Credit

Now that the Character templates are not tied to a specific model, it should now be possible to assign very different models to each of the characters and allow them to converse.Z

In [44]:
# Load a secondary model
# This is being loaded to my second GPU.
user_model = rp.CausalLM(
    "/home/dinalt/rust/models/mistralai_Mistral-7B-Instruct-v0.2",
    #"/home/dinalt/rust/models/walsh_rpbuild",
    device=1,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
)

No PAD token defined. Setting pad token to EOS


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNor

In [45]:
# Select roles and write a scenario outline

transformers.set_seed(42)
example_char = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
example_user = rp.char.CharMeta.from_data(rp.data.random_char(dataset))
rp.data.print_char_summary(example_char, example_user)

# Write a script
writer = rp.writer.Writer(causal_lm, debug_level=1)
script = writer(example_char, example_user)
print(script)

Char: Valentina Rossi

Char Summary: A former war correspondent turned politician, Valentina Rossi brings a unique perspective and a deep understanding of global issues to the political arena. Her charisma and passion inspire others to work towards a more peaceful and just world.

Proxy User: Isabella Snowsong

Proxy User Summary: A skilled hunter and tracker, Isabella Snowsong possesses an intimate connection with the Arctic wildlife. Her ability to communicate with animals grants her invaluable insights and assistance during her explorations.

Title: "The Call of the Arctic: A Journey towards Unity"

Plot Outline:
The story begins with Isabella Snowsong reflecting on her spiritual journey in the Arctic wilderness, where she seeks guidance from the very wildlife she shares her life with. While spending time in her igloo home, she reads an article written by Valentina Rossi, a Mediterranean politician pushing for greater diplomacy and understanding among nations. Isabella is captivated

In [46]:
transformers.set_seed(42)
#dialog_filter = DialogFilter(causal_lm, debug=True)
dialog_filter=None

char = rp.char.Character(
    char_meta=example_char,
    causal_lm=causal_lm,
    generation_config="Midnight-Enigma",
    user_meta=example_user,
    template_config=rp.char.TemplateConfig(),
    gen_post_process=dialog_filter,
    # Shows character prompts and internal diagnostics
    debug=False,
    history_token_limit=1800
)

proxy_user = rp.char.Character(
    char_meta=example_user,
    causal_lm=user_model,
    generation_config="Midnight-Enigma",
    user_meta=example_char,
    template_config=rp.char.TemplateConfig(
        chat_template=load_template("models/mistral.jinja"),
        instruct=load_template("dialog_instruct_simple.jinja"),
    ),
    gen_post_process=dialog_filter,
    debug=False,
    history_token_limit=1800
)

director = rp.director.Director(
    causal_lm,
    script=script,
    # Shows director prompts and more...
    debug=False,
    history_token_limit=2000
)

roleplay = rp.roleplay.Roleplay(
    char=char,
    user=proxy_user,
    scenario=script,
    director=director,

    # Shows generated dialog and control events
    debug=True
)

In [47]:
conversations = roleplay(2000)

-------------------------- user:Valentina Rossi (220)---------------------------
Valentina Rossi enters the grand conference room, her confident strides echoing off the marble walls. As she scans the sea of faces before her, her bright hazel eyes land on yours. A warm smile graces her lips, revealing the slightest dimple on each cheek.

*"Hello there,"* Valentina begins, extending a slender hand adorned with intricately designed bangles. *Her voice carries the weight of experience and passion.* It's a pleasure to see you again, Isabella Snowsong. I must confess, I wasn't expecting to run into you at this gathering. Settling into a plush chair, she continues, The issues we'll discuss here today hold great importance not only for our nation, but for the entire global community. May we approach them with open minds and hearts, remembering that every action, no matter how small, has the power to create ripples of change. Let us work together to bridge divides and forge a path towards a mor

In [48]:
proxy_user.print_conversation(director_log=True)

----------------------------- system:system (1076)------------------------------
Name: Isabella Snowsong
Polar bear shaman (part human, part polar bear)
White fur covering entire body except for her bright blue eyes and dark brown pupils
Long, curved claws on all four paws
Slightly taller than average height for a polar bear, around 6'3"
Early 40s

Isabella Snowsong is a revered member of the Arctic community, known for her unparalleled skills as a hunter and tracker. Born with a unique gift to communicate with Arctic wildlife, she has spent her life learning to harness its power to aid her in exploration and survival. With a deep respect and appreciation for the natural world, she lives in harmony with the harsh Arctic environment.

Isabella exudes a calming presence, with a gentle yet strong demeanor. Her wisdom comes from countless hours spent observing the intricacies of Arctic ecosystems and understanding the behavior patterns of various creatures. She is patient, compassionate, a